<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Загрузка-данных" data-toc-modified-id="Загрузка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Загрузка данных</a></span></li><li><span><a href="#Умножение-матриц" data-toc-modified-id="Умножение-матриц-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Умножение матриц</a></span></li><li><span><a href="#Алгоритм-преобразования" data-toc-modified-id="Алгоритм-преобразования-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Алгоритм преобразования</a></span></li><li><span><a href="#Проверка-алгоритма" data-toc-modified-id="Проверка-алгоритма-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Проверка алгоритма</a></span><ul class="toc-item"><li><span><a href="#Вывод" data-toc-modified-id="Вывод-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Вывод</a></span></li></ul></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Защита персональных данных клиентов

Вам нужно защитить данные клиентов страховой компании «Хоть потоп». Разработайте такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуйте корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

## Загрузка данных

In [1]:
# импорт библиотек

import pandas as pd
import numpy as np
from numpy.linalg import inv, det
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

In [2]:
# открытие файла - локальное или на платформе
try:
    data = pd.read_csv('insurance.csv')
except:
    data = pd.read_csv('/datasets/insurance.csv')

In [3]:
data.head()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


Пропусков в датафрейме нет.

**Признаки**: пол, возраст, зарплата, количество членов семьи.

**Целевой признак**: страховые выплаты (количество за последние 5 лет).

Смотрим дубликаты и уникальные значения на предмет аномалий.

In [5]:
data.duplicated().sum()

153

In [6]:
for col in data.columns:
    print(col)
    print(data[col].unique())
    print()

Пол
[1 0]

Возраст
[41. 46. 29. 21. 28. 43. 39. 25. 36. 32. 38. 23. 40. 34. 26. 42. 27. 33.
 47. 30. 19. 31. 22. 20. 24. 18. 37. 48. 45. 44. 52. 49. 35. 56. 65. 55.
 57. 54. 50. 53. 51. 58. 59. 60. 61. 62.]

Зарплата
[49600. 38000. 21000. 41700. 26100. 41000. 39700. 38600. 49700. 51700.
 36600. 29300. 39500. 55000. 43700. 23300. 48900. 33200. 36900. 43500.
 36100. 26600. 48700. 40400. 38400. 34600. 34800. 36800. 42200. 46300.
 30300. 51000. 28100. 64800. 30400. 45300. 38300. 49500. 19400. 40200.
 31700. 69200. 33100. 31600. 34500. 38700. 39600. 42400. 34900. 30500.
 24200. 49900. 14300. 47000. 44800. 43800. 42700. 35400. 57200. 29600.
 37400. 48100. 33700. 61800. 39400. 15600. 52600. 37600. 52500. 32700.
 51600. 60900. 41800. 47400. 26500. 45900. 35700. 34300. 26700. 25700.
 33300. 31100. 31500. 42100. 37300. 42500. 27300. 46800. 33500. 44300.
 41600. 53900. 40100. 44600. 45000. 32000. 38200. 33000. 38500. 51800.
 33800. 46400. 43200. 31800. 50200. 35100. 30700. 45800. 49300. 42800.
 3

Аномалий не обнаружено, можно удалить явные дубликаты (ибо полная копия всех значений для разных клиентов маловероятна, следовательно, такие значения повлияют на качество модели в будущем) и выполнить приведение типов к целому в столбцах "Возраст" и "Зарплата" (т.к. дробных значений там нет).

In [7]:
data = data.drop_duplicates()

data['Возраст'] = data['Возраст'].astype('int64')
data['Зарплата'] = data['Зарплата'].astype('int64')
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4847 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype
---  ------             --------------  -----
 0   Пол                4847 non-null   int64
 1   Возраст            4847 non-null   int64
 2   Зарплата           4847 non-null   int64
 3   Члены семьи        4847 non-null   int64
 4   Страховые выплаты  4847 non-null   int64
dtypes: int64(5)
memory usage: 227.2 KB


In [8]:
data.head()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41,49600,1,0
1,0,46,38000,1,1
2,0,29,21000,0,0
3,0,21,41700,2,0
4,1,28,26100,0,0


## Умножение матриц

В этом задании вы можете записывать формулы в *Jupyter Notebook.*

Чтобы записать формулу внутри текста, окружите её символами доллара \\$; если снаружи —  двойными символами \\$\\$. Эти формулы записываются на языке вёрстки *LaTeX.* 

Для примера мы записали формулы линейной регрессии. Можете их скопировать и отредактировать, чтобы решить задачу.

Работать в *LaTeX* необязательно.

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

**Вопрос:** Признаки умножают на обратимую матрицу P: $Z = X@P$. Изменится ли качество линейной регрессии?

**Ответ:** не изменится.

**Обоснование:**

Докажем, что
$$
a = Xw = Zw_1 = XPw_1 = a_1
$$

Чтобы это равенство выполнялось, необходимо соблюсти условие:

$$
w_1 = P^{-1}w
$$

Если заменить X на Z, получим такое:

$$
a_1 = Zw_1
$$
$$
w_1 = (Z^T Z)^{-1} Z^Ty
$$

Раскрываем Z и получаем:
$$
a_1 = XPw_1
$$

$$
w_1 = ((XP)^T XP)^{-1} (XP)^Ty
$$

При этом:
$$
w = (X^T X)^{-1} X^T y
$$

Находим квадратные матрицы, чтоб можно было воспользоваться произведением трёх множителей $(ABС)^{-1} = ((AB)С)^{-1} = ((AB)С)^{-1} = С^{-1}(AB)^{-1} = С^{-1}B^{-1}A^{-1}$

$$
w_1 = (P^T (X^T X) P)^{-1} P^T X^T y
$$

После раскрытия скобок получаем:

$$
w_1 = P^{-1} (X^T X)^{-1} (P^T)^{-1} P^T X^T y
$$

Так как $P$ по условию обратимая, то произведение $(P^T)^{-1} P^T$ равно единичной матрице.

$$
w_1 = P^{-1} (X^T X)^{-1} E X^T y = P^{-1} (X^T X)^{-1} X^T y
$$

Здесь справа получается формула $w = (X^T X)^{-1} X^T y$

$$
w_1 = P^{-1} (X^T X)^{-1} X^T y = P^{-1} w
$$

Новое значение весов подставляем в исходную формулу для предсказаний $a = Xw$:

$$
a_1 = Zw_1 = XPP^{-1} w
$$

$P$ по условию обратима, а значит, произведение $PP^{-1}$ равно единичной матрице $E$:

$$
a_1 = XPP^{-1} w = XEw = Xw = a
$$

Или коротко:

$$
a = Xw = Z(P^{-1}w) = XP(P^{-1}w) = XEw = Xw = a
$$

Также из этих вычислений можно заключить, что, хотя предсказания и не изменятся, но изменятся веса "зашифрованных" признаков, а следовательно, модель нужно будет обучить на них заново (модель, обученная на оригинальных признаках, не подойдёт для предсказания по зашифрованным данным, и наоборот).

## Алгоритм преобразования

**Алгоритм**

Он будет заключаться в следующем: создадим случайную обратимую матрицу P, затем умножим матрицу с признаками X на матрицу P.

$$
Z = X @ P
$$

Для возврата к исходной матрице $X$, умножим новую матрицу $Z$ на обратную матрицу $P^{-1}$.

$$
X = Z @ P^{-1}
$$

**Обоснование**

Качество линейной регрессии не изменится при умножении матрицы признаков на любую обратимую матрицу P. Поэтому преобразование по данному алгоритму не приведёт к иному результату линейной регрессии.

## Проверка алгоритма

Формируем признаки и цель, создаём матрицу признаков и случайную матрицу.

In [9]:
features = data.drop('Страховые выплаты', axis=1)
target = data['Страховые выплаты']

n = features.shape[1] # число строк и столбцов для случайной матрицы

features_matrix = features.values
# random_matrix = np.random.randint(1, 10, (n, n)) # старый код

# создание случайной обратимой матрицы
def get_rand_matrix(n):
    det = 0
    while det == 0:
        matrix = np.random.normal(size=(n, n))
        det = np.linalg.det(matrix)
    return matrix

random_matrix = get_rand_matrix(n)

In [10]:
cipher_matrix = features_matrix @ random_matrix
features_cipher = list(cipher_matrix)
features_cipher

[array([-11469.07194396,  67228.32284347, -25564.59935733, -53554.73390306]),
 array([ -8786.0852017 ,  51520.091061  , -19558.13147491, -41006.76392699]),
 array([ -4854.77267174,  28475.321265  , -10801.26341718, -22655.69404373]),
 array([ -9644.42726015,  56507.41453631, -21519.74826609, -45047.3655835 ]),
 array([ -6034.14790476,  35382.43341367, -13439.47388161, -28170.24712422]),
 array([ -9480.7051487 ,  55580.1926308 , -21114.98749745, -44254.59050005]),
 array([ -9180.33195795,  53815.28186228, -20450.58712988, -42855.65306188]),
 array([ -8928.51697886,  52310.81622616, -19909.87437057, -41689.65773612]),
 array([-11492.56361031,  67358.90264487, -25625.93667737, -53670.90969839]),
 array([-11955.4147157 ,  70064.2355081 , -26667.66075754, -55839.50678604]),
 array([ -8463.59448095,  49602.64936705, -18874.34010149, -39526.62981667]),
 array([ -6773.51687716,  39726.97937086, -15074.62443011, -31613.53716621]),
 array([ -9136.25303924,  53528.84773068, -20378.98382001, -4266

После преобразования исходный набор данных не определяется.

In [11]:
features_train_origin, features_test_origin, target_train_origin, target_test_origin = train_test_split(
    features, target, test_size=0.25, random_state=123)
features_train_cipher, features_test_cipher, target_train_cipher, target_test_cipher = train_test_split(
    features_cipher, target, test_size=0.25, random_state=123)

In [12]:
model1 = LinearRegression()
model2 = LinearRegression()

model1.fit(features_train_origin, target_train_origin)
predicted_origin = model1.predict(features_test_origin)
print("R2 исходного набора данных:", r2_score(predicted_origin, target_test_origin))

R2 исходного набора данных: -0.2447353824307128


In [13]:
model2.fit(features_train_cipher, target_train_cipher)
predicted_cipher = model2.predict(features_test_cipher)
print("R2 зашифрованного набора данных:", r2_score(predicted_cipher, target_test_cipher))

R2 зашифрованного набора данных: -0.24473538243077586


Метрика R2 в значимых пределах не изменилась, следовательно, качество модели при преобразовании признаков осталось прежним. Попробуем восстановить исходную матрицу по формуле $X = Z @ P^{-1}$

In [14]:
deciphered_matrix = cipher_matrix @ inv(random_matrix)

# сравнение округлённой расшифрованной матрицы с оригинальной; если все значения совпадут, выведет True
(np.round(deciphered_matrix) == features_matrix).all()

True

### Вывод
Матрицы одинаковы, следовательно, мы реализовали алгоритм, позволяющий сначала скрыть данные с целью их деанонимизации, а затем восстановить, используя случайшую обратимую матрицу. Качество линейной регрессии осталось неизменным.

## Чек-лист проверки

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  Jupyter Notebook открыт
- [x]  Весь код выполняется без ошибок
- [x]  Ячейки с кодом расположены в порядке исполнения
- [x]  Выполнен шаг 1: данные загружены
- [x]  Выполнен шаг 2: получен ответ на вопрос об умножении матриц
    - [x]  Указан правильный вариант ответа
    - [x]  Вариант обоснован
- [x]  Выполнен шаг 3: предложен алгоритм преобразования
    - [x]  Алгоритм описан
    - [x]  Алгоритм обоснован
- [x]  Выполнен шаг 4: алгоритм проверен
    - [x]  Алгоритм реализован
    - [x]  Проведено сравнение качества моделей до и после преобразования